In [16]:
# import os + os.chir is used to get to the llms directory to use the usermodules 
import os
os.chdir('C:/Users/AViey/OneDrive/Documents/College/UCI/Stats/stats170a/copilot/copilot/llms/user_modules') 

from  table_instances import Summaries, Sentiments, Base
from  db import Session, get_training_data
from helper_functions import compute_summary_scores
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

session = Session()

In [17]:
gemma = pd.read_csv("C:/Users/AViey/OneDrive/Documents/College/UCI/Stats/stats170a/copilot/copilot/Datasets/gemma_summary_results.csv")
gemma.head()

,dialogue_id,Time Taken (s),GPU Usage (MB),Summary
0,3816,0.633181,750.495742,The dialogue is about a person ordering a suit...
1,3,0.633181,750.495742,Two people are talking and searching for missi...
2,4,0.633181,750.495742,"The dialogue is between two people, #Person1# ..."
3,5,0.633181,750.495742,"The dialogue is between two people, Malik and ..."
4,6,0.633181,750.495742,"The dialogue is between two people, Lisa and A..."


In [18]:
training_data = get_training_data()
training_data.head()

,dialogue_id,dataset,dialogue_text,actual_summary,actual_sentiment
0,3816,training,"#Person1#: Good morning, sir. Is there anythin...",#Person2# wants a suit made in 100 % black Bar...,neutral
1,3,training,"#Person1#: Excuse me, did you see a set of key...",#Person1#'s looking for a set of keys and asks...,positive
2,4,training,#Person1#: Why didn't you tell me you had a gi...,#Person1#'s angry because #Person2# didn't tel...,negative
3,5,training,"#Person1#: Watsup, ladies! Y'll looking'fine t...",Malik invites Nikki to dance. Nikki agrees if ...,positive
4,6,training,"#Person1#: Happy birthday, Aims!\n#Person2#: T...",Lisa gives Aims a birthday present and Aims lo...,positive


In [19]:
merged_df = training_data[["dialogue_id","actual_summary"]]
merged_df["generated_summary"] = gemma["Summary"] 
merged_df.head()

C:\Users\AViey\AppData\Local\Temp\ipykernel_8980\2802136808.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df["generated_summary"] = gemma["Summary"]


,dialogue_id,actual_summary,generated_summary
0,3816,#Person2# wants a suit made in 100 % black Bar...,The dialogue is about a person ordering a suit...
1,3,#Person1#'s looking for a set of keys and asks...,Two people are talking and searching for missi...
2,4,#Person1#'s angry because #Person2# didn't tel...,"The dialogue is between two people, #Person1# ..."
3,5,Malik invites Nikki to dance. Nikki agrees if ...,"The dialogue is between two people, Malik and ..."
4,6,Lisa gives Aims a birthday present and Aims lo...,"The dialogue is between two people, Lisa and A..."


In [20]:
merged_df = merged_df.apply(compute_summary_scores, axis=1)
merged_df.head()

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

,dialogue_id,actual_summary,generated_summary,rouge_1,rouge_2,rouge_l,bert_precision,bert_recall,bert_f1,meteor
0,3816,#Person2# wants a suit made in 100 % black Bar...,The dialogue is about a person ordering a suit...,0.468085,0.259259,0.468085,0.893794,0.931543,0.912278,0.669124
1,3,#Person1#'s looking for a set of keys and asks...,Two people are talking and searching for missi...,0.269231,0.053333,0.269231,0.867384,0.865039,0.866210,0.270274
2,4,#Person1#'s angry because #Person2# didn't tel...,"The dialogue is between two people, #Person1# ...",0.250000,0.043011,0.222222,0.862367,0.899115,0.880357,0.455599
3,5,Malik invites Nikki to dance. Nikki agrees if ...,"The dialogue is between two people, Malik and ...",0.156863,0.027778,0.156863,0.856515,0.872255,0.864313,0.231092
4,6,Lisa gives Aims a birthday present and Aims lo...,"The dialogue is between two people, Lisa and A...",0.341463,0.000000,0.243902,0.873825,0.910648,0.891857,0.306122


In [21]:
merged_df.to_csv("../../Datasets/gemma_sample_scores.csv", index = False)